In [1]:
import numpy as np
import matplotlib.pyplot as plt

from scipy.sparse import csc_matrix

from pixelcorr2D import get_distances, get_intensities

import h5py
from time import time

In [4]:
# detector parameters
det_params = {}
det_params['mu'] = 3.445930 * 10**-3 #in 1/micron
det_params['I0'] = 1 
    
#pixel dimentions
det_params['pl'] = 75 #pixel length, in micrometers 
det_params['pw'] =  450 #pixel width, in micrometers
det_params['ph'] = 75 #pixel height, in micrometers
    
#detector dimentions
det_params['det_numpixels'] = (100, 100) #number of rows in pixels, number of columns in pixels
#detector limits in Cartesian coordinates
det_params['det_xlim'] = (0.0, det_params['pw'])
det_params['det_ylim'] = (0.0, det_params['det_numpixels'][1] * det_params['pl'])
det_params['det_zlim'] = (0.0, det_params['det_numpixels'][0] * det_params['ph'])
    
#Cartesian coordinates of the sample, given by the poni file 
det_params['sx'] = -0.4051247560395305 * 10 **4 # -dist in poni file, in micrometers
det_params['sy'] = 0.17601077769492505 * 10 **4 # P2 in poni file, in micrometers
det_params['sz'] = 0.02184240399163788 * 10 **4 # P1 in poni file, in micrometers

det_params['j'] = 10 #planes

In [5]:
# preallocate linear storage
'''
nsz = np.prod(det_params['det_dims']) * 5 #number of pixels times intersections per pixel, although then he truncates buffer...
data = np.zeros((nsz,), dtype=np.float64) #vertical vector 
row_ind = np.zeros((nsz,), dtype=np.int32)
col_ind = np.zeros((nsz,), dtype=np.int32)
iend = 0
'''
#all rays
hit_col, hit_row = meshgrid(linspace(0, 100, 10), linspace(0, 100, 10)) 
hit_row = hit_row.flatten()
hit_col = hit_col.flatten()

recalc_geometry = True

t_start_geom_calc = time()

#_det_row, _det_col are LISTS OF INTEGERS!! and they are columns!!!
#they contain the pixels for all rays, which means that most probably they will have repeated values!!
#data is the absorbed intensity per each pixel pear each ray

'''
if recalc_geometry:
    # pass over each pixel and calculate the absorption coefficients
    for irow in range(det_params['det_dims'][0]):
        for icol in range(det_params['det_dims'][1]):
            # get distancies and intensities
            _dist, _det_row, _det_col = get_distances(irow, icol, det_params)
            _ti, _ai = get_intensities(_dist, det_params)
            nlen = len(_dist)
            row_ind[iend:(iend+nlen)] = np.array(_det_row) * det_params['det_dims'][1] + np.array(_det_col)
            col_ind[iend:(iend+nlen)] = irow * det_params['det_dims'][1] + icol
            data[iend:(iend+nlen)] = _ai
            iend = iend + nlen
    # truncate buffers
    row_ind = row_ind[:iend]
    col_ind = col_ind[:iend]
    data = data[:iend]
else:
    # load data from file
    with h5py.File('G_matrix.h5','r') as h5f:
        row_ind = h5f['/G/row_ind'][()]
        col_ind = h5f['/G/col_ind'][()]
        data = h5f['/G/data'][()]
'''

if recalc_geometry:
    _dist, row_ind, col_ind = get_distances(hit_row, hit_col, det_params)
    _ti, data = get_intensities(_dist, det_params)
else:
    # load data from file
    with h5py.File('G_matrix.h5','r') as h5f:
        row_ind = h5f['/G/row_ind'][()]
        col_ind = h5f['/G/col_ind'][()]
        data = h5f['/G/data'][()]

t_finished_geom_calc = time()
        
# forward propagation matrix
t_start_matrix_create = time()
nsz = np.prod(det_params['det_dims']) #number of pixels
G = csc_matrix((data,(row_ind, col_ind)),shape=(nsz,nsz))
t_finished_matrix_create = time()

# print statistics
print('forward propagation matrix (recalc=%d)' % (recalc_geometry,))
print('shape: ', G.shape)
print('nnz: %d, nnz/pixel: %.1f' % (G.nnz, G.nnz/G.shape[0],))
print('total absorption: %.1f, absorption/pixel: %.2f' % (G.sum(), G.sum()/G.shape[0],))
print('geom calculation took: %.3f sec' % (t_finished_geom_calc-t_start_geom_calc,))
print('matrix creation took: %.3f sec' % (t_finished_matrix_create-t_start_matrix_create,))

# save matrix
with h5py.File('G_matrix.h5','w') as h5f:
    # create dataset
    h5f.create_dataset('/G/shape', data=np.array(G.shape))
    h5f.create_dataset('/G/row_ind', data=row_ind)
    h5f.create_dataset('/G/col_ind', data=col_ind)
    h5f.create_dataset('/G/data', data=data)

#print(G)

forward propagation matrix (recalc=1)
shape:  (1000000, 1000000)
nnz: 3153420, nnz/pixel: 3.2
total absorption: 801804.4, absorption/pixel: 0.80
geom calculation took: 417.709 sec
matrix creation took: 0.104 sec


In [6]:
# small matrix printout
row = 3; col = 900;
print(G[:,row*det_params['det_dims'][1]+col])
print('\n')
row = 999; col = 100;
print(G[:,row*det_params['det_dims'][1]+col])

  (901, 0)	0.05729021713675053
  (1901, 0)	0.1839896474925652
  (2900, 0)	0.2637072672745873
  (2901, 0)	0.06786446255809092
  (3900, 0)	0.25510208462952955


  (999099, 0)	0.31087318270018144
  (999100, 0)	0.48340476591392456
